In [13]:
import os
from Bio import SeqIO
import io
from Bio.Seq import Seq
import subprocess

In [14]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [15]:
def make_raw_files_for_alignment(gene_sequence,annotated_genome_location,annotated_species_name,error_exon,query_fasta_sequence,query_length):
    for offset in range(3):
        translated_sequence = str(gene_sequence[offset:].translate()).split("*")
        for i in range(len(translated_sequence)):
            if len(translated_sequence[i])> 0.8*query_length:
                sequence_set = f">set{i+1}_frame{offset}\n{translated_sequence[i]}\n\n"
                # print(i+1, offset)
                
                with open(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment/{error_exon}_translated_genomic_sequence_{i+1}_frame{offset}.fa",'w') as out_file:
                    output = f"{query_fasta_sequence}\n\n{sequence_set}"
                    out_file.write(output)

In [16]:
def run_mafft(annotated_genome_location,annotated_species_name,error_exon):
    location = f'{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment'
    list_of_files_to_run_mafft_on = os.listdir(location)
    for file in list_of_files_to_run_mafft_on:
        if file.endswith(".fa"):
            command = f'"mafft" --localpair --maxiterate 16 --reorder --distout "{location}/{file}" > "{location}/alignment_{file}.txt"'
            # print(command)
            subprocess.run(f'{command}', shell=True, stderr = subprocess.DEVNULL) 
            # os.system(f'{command}')
            command = f'"mafft" --localpair --clustalout --maxiterate 16 --reorder "{location}/{file}" > "{location}/alignment_clustal_{file}.txt"'
            subprocess.run(f'{command}', shell=True, stderr = subprocess.DEVNULL) 
            
            # os.system(f'{command}')
    return(location)

In [17]:
def process_mafft_output(mafft_run_folder, error_exon):
    list_of_files_in_mafft_run_folder = os.listdir(mafft_run_folder)
    score_output = []
    score = 99
    min_score_sequence = ''
    alignment_file = ''
    for file in list_of_files_in_mafft_run_folder:
        if file.endswith(".fa.hat2"):
            with io.open(f"{mafft_run_folder}/{file}", 'r') as dist_matrix_file:
                dist_matrix_list = dist_matrix_file.readlines()
            
            sequence_name = dist_matrix_list[-2].rstrip().split("=")[1]
            distance_score = float(dist_matrix_list[-1].rstrip())
            if len(score_output) < 5:
                score_output.append( [sequence_name,distance_score])
            else:
                for i in range(len(score_output)):
                    score_at_this_index = score_output[i][1]
                    if distance_score < score_at_this_index:
                        score_output[i] = [sequence_name,distance_score]
                        break
            if distance_score < score:
                score = distance_score
                min_score_sequence = sequence_name
                alignment_file = f'alignment_{file.replace(".hat2",".txt")}'
    # print(score_output)
    print(f"min = {min_score_sequence}, {score}" )
    # score_out_merged = '\n'.join(score_output)
    print(f"5 top scores:\n{score_output}")
    print(f"{mafft_run_folder}/{alignment_file}")
    
    alignment_file = SeqIO.parse(f"{mafft_run_folder}/{alignment_file}", 'fasta')
    # print (records.id)
    fasta_start_position = 0
    fasta_end_position = 0
    start_switch = 0
    end_switch = 0
    alignment_name = ''
    for records in alignment_file:
        
        
        
        if error_exon in records.id:
            gap_counter = 0
            base_counter = 0
            for current_position in range(len(records.seq)):
                sequence_length = len(records.seq) - records.seq.count('-')
                # print(f"fasta_end_position {fasta_end_position} fasta_start_position {fasta_start_position}")
                # print(f"current_position = {current_position}, {len(records.seq)}")
                # print(records.seq[current_position])
                # print("fasta_start_position",fasta_start_position)
                # print("start_switch",start_switch)
                
                # print(gap_counter, base_counter)
                # print(5,0.2*sequence_length)
                if start_switch == 1 and records.seq[current_position] == "-" and gap_counter > 3 and (len(records.seq[:current_position]) - records.seq[:current_position].count('-')) < (0.1*sequence_length):
                #  :
                    # print("\n\nhere\n\n")
                    start_switch = 0
                    gap_counter = 0

                if "-" not in records.seq[current_position] and start_switch == 0 :
                    fasta_start_position = current_position
                    start_switch = 1
                
                    
                
                    
                if end_switch == 1 and "-" not in records.seq[current_position] and (len(records.seq[current_position:]) - records.seq[current_position:].count('-')) > 0.1*sequence_length :
                    end_switch = 0
                    gap_counter = 0
                
                    
                if start_switch == 1 and records.seq[current_position] == "-" and end_switch == 0:
                    # print(f"base_counter {base_counter}")
                    # print(f"fasta_end_position {fasta_end_position}")
                    
                    # print(f"fasta_end_position {fasta_end_position}")
                    
                    fasta_end_position = current_position
                    end_switch = 1
                if "-" in records.seq[current_position]:
                    gap_counter += 1
                else:
                    gap_counter = 0
                    base_counter += 1
        else:
            alignment_name = records.id
            if fasta_end_position == 0:
                
                fasta_end_position = current_position
            end_switch = 1
            start_switch = 1

        if start_switch == 1 and end_switch == 1:
            print(fasta_start_position, fasta_end_position)
            gene_sequence = records.seq[fasta_start_position:fasta_end_position]
            print(f"{records.id}\n{gene_sequence}")
    return(gene_sequence, alignment_name)

# process_mafft_output(mafft_run_folder, error_exon)

In [18]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_annotated_genome_name(annotated_genome_location, species):
    list_of_annotated_genomes = os.listdir(annotated_genome_location)
  
    # print(annotated_species)
    for annotated_species in list_of_annotated_genomes:
        if annotated_species.endswith(species):
            return(annotated_species)
             
    if annotated_species_name == '':
        print(f"Error with annotated species name")
        assert False

def check_and_make_folders(annotated_genome_location,annotated_species_name,error_exon):
    list_of_files_inside_annotated_species_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}")

    if f"Period_gene_genomic_sequence_individual_exon_{query_species}" not in list_of_files_inside_annotated_species_folder:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}")
    
    list_of_exon_directories = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/")
    
    if error_exon not in list_of_exon_directories:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}")
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment")
    elif error_exon in list_of_exon_directories:
        list_of_folders_1 = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}")
        if "for_alignment" not in list_of_folders_1:
            os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment")
        list_of_files = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment")
        for file in list_of_files:
            os.remove(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment/{file}")


In [19]:
def get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name):
    from Bio import SeqIO
    print("Getting Gene")
    list_of_files_inside_annotated_species_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}")
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"Period_gene_genomic_sequence_individual_exon_{query_species}" not in list_of_files_inside_annotated_species_folder:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}")

    list_of_files_inside_indiv_exon_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}")
    if "temp" not in list_of_files_inside_indiv_exon_folder:
         os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp")

    # os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}"')
    # subprocess.run("pwd")
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
    # os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp/temp_genome.fa"')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    genome = SeqIO.parse(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)

In [20]:
def mafft_process(previous_exon_coordinates,
                  next_exon_coordinates,
                  current_exon_coordinates,
                  query_species,
                  query_transcript,
                  query_location,
                  annotated_genome_location,
                 genome_location):
    query_exon = current_exon_coordinates[6].split("query")[-1][1:]
    error_exon = query_exon
    # print("AA")
    genome_file = get_genome_file(genome_location,species)
    
    upstream_exon_line, downstream_exon_line = previous_exon_coordinates.split(","),next_exon_coordinates.split(",")
    complement,scaffold = upstream_exon_line[4], upstream_exon_line[1]

    if complement == "0":
        gene_start = min(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
        gene_end = max(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
    
    if complement == "1":
        gene_start = min(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
        gene_end = max(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
    
    
    with open(f"{query_location}/{query_species}/{query_transcript}/query_{query_exon}.fa", 'r') as query_file:
        query_file_list = query_file.readlines()
        query_fasta_sequence = "".join(query_file_list)
        left_overhang = query_file_list[0].split("Frame")[1][1]
        right_overhang = query_file_list[0].split("rightoh")[1][1]
        original_query_name = query_file_list[0]
        original_query = query_file_list[1]
        
    query_length = len(query_fasta_sequence.split("\n")[1])
    
    annotated_species_name = get_annotated_genome_name(annotated_genome_location, species)
    # print(annotated_species_name)

    # print(annotated_genome_location)
    gene_sequence = get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name)
    # assert False
    check_and_make_folders(annotated_genome_location,annotated_species_name,error_exon)
    
    
    make_raw_files_for_alignment(gene_sequence,annotated_genome_location,annotated_species_name,error_exon,query_fasta_sequence,query_length)
    
    return(annotated_species_name, error_exon,species, genome_file,gene_sequence,left_overhang,right_overhang,scaffold, original_query_name,original_query )


In [23]:

blast_output_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result"
# blast_output_location = "/mnt/j/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine/1.Blast_result"
species_list = os.listdir(blast_output_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
species_list.remove("Hestina_assimilis")
species_list.remove("Charaxes_varanes")
for species in species_list:
    annotated_genome_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result"
    query_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Query"
    list_of_query_species = os.listdir(query_location)
    if "desktop.ini" in list_of_query_species:
        list_of_query_species.remove("desktop.ini")
    
    list_of_query_species = ["29.Nymphalis_io"]
    for query_species in list_of_query_species:
        list_of_query_transcripts = os.listdir(f"{query_location}/{query_species}")
        list_of_query_transcripts.remove("desktop.ini")
        for query_transcript in list_of_query_transcripts:


            genome_location = "/mnt/f/Genomes_2023-12-26"



            output_coordinate_file = "Species," + "Scaffold," + "Start," + "Stop," + "Complement," + "Error," + "Gene,"+ "Query_start," + "Query_stop,"+ "Query_Length," +  "AG_GT," + "Spliceator_prediction\n"
            exons_to_check_list = ["Exon_2","Exon_3","Exon_4","Exon_5","Exon_6","Exon_7","Exon_8","Exon_9","Exon_10","Exon_11","Exon_12","Exon_13","Exon_14","Exon_15","Exon_16","Exon_17","Exon_18","Exon_19","Exon_20","Exon_21","Exon_22","Exon_23","Exon_24","Exon_25"]
#             exons_to_check_list = ["Exon_21"]

            list_of_files_in_species_folder = os.listdir(f"{blast_output_location}/{species}")
            coordinate_file_name = ''
        #     print(list_of_files_in_species_folder)
            for file_names in list_of_files_in_species_folder:

                if file_names.endswith("_coordinates_old.csv"):
                    coordinate_file_name = file_names
            if coordinate_file_name =='':
                print(f"Coordinate file error")
                assert False

            with io.open(f"{blast_output_location}/{species}/{coordinate_file_name}", 'r') as temp_file_open:
                coordinate_file_lines = temp_file_open.readlines()
            # print(coordinate_file_lines[0])
            for exons_to_check in exons_to_check_list:

                for i in range(1,len(coordinate_file_lines)):
                    coordinate_file_lines_split = coordinate_file_lines[i].split(",")
                    current_exon_coordinates = coordinate_file_lines[i].split(",")
                    if i == 1:
                        scaffold_first_exon = coordinate_file_lines[i].split(",")[1]
                        first_exon_start, first_exon_end = coordinate_file_lines[i].split(",")[2],coordinate_file_lines[i].split(",")[3]
                    # if coordinate_file_lines_split[5] == "Y":
                    if  coordinate_file_lines_split[6].endswith(exons_to_check):
                        if i == 1:
                            print(f'First Exon has errors\n{coordinate_file_lines[i]}')
                            assert False
                        else:
                            print(f"Scaffold = {scaffold_first_exon}, start = {first_exon_start}, end = {first_exon_end}")
                            print(coordinate_file_lines[i])
                            # process_current_exon = input("Error Found! Process?")
                            process_current_exon = 'y'
                            if process_current_exon.lower()[0] == "y":
                                if i>2:
                                    previous_exon_number = i-2
                                else:
                                    previous_exon_number = i-1
                                if i+2 <=int(exons_to_check_list[-1].split("_")[1]):
                                    next_exon_number = i+2
                                else:
                                    next_exon_number = i+1
                                if i >= int(exons_to_check_list[-1].split("_")[1]):
                                    next_exon_number = int(exons_to_check_list[-1].split("_")[1])
#                                     continue

                                '''
                                the above code ensures that missing exons do not cause issues
                                '''

                                previous_exon_coordinates = ''
                                next_exon_coordinates = ''
                                while True:
                                    print(previous_exon_number)
                                    if coordinate_file_lines[previous_exon_number].split(",")[5] == "N":
                                        previous_exon_coordinates = coordinate_file_lines[previous_exon_number]
                                        break
                                    # else:
                                    #     proceed_prompt = input(f"Previous exon coordinate\n{coordinate_file_lines[previous_exon_number]}\nProceed?")
                                    #     if proceed_prompt.lower()[0] == "y":
                                    #         previous_exon_coordinates = coordinate_file_lines[previous_exon_number]
                                    #         break
                                    previous_exon_number -= 1
                                while True:
                                    print(next_exon_number)
                                    try:
                                        if coordinate_file_lines[next_exon_number].split(",")[5] == "N":
                                            next_exon_coordinates = coordinate_file_lines[next_exon_number]
                                            break
                                    except:
                                        if coordinate_file_lines[next_exon_number-1].split(",")[5] == "N":
                                            next_exon_coordinates = coordinate_file_lines[next_exon_number-1]
                                            break
                                   
                                    next_exon_number += 1


                                print(previous_exon_coordinates)
                                print(next_exon_coordinates)
                                annotated_species_name, error_exon, species, genome_file, gene_sequence,left_overhang,right_overhang,scaffold,original_query_name, original_query = mafft_process(previous_exon_coordinates,
                                                                                                                                  next_exon_coordinates,
                                                                                                                                  current_exon_coordinates,
                                                                                                                                  query_species,
                                                                                                                                  query_transcript,
                                                                                                                                  query_location,
                                                                                                                                  annotated_genome_location,
                                                                                                                                     genome_location)

                                mafft_run_folder = run_mafft(annotated_genome_location,annotated_species_name,error_exon)

                                possible_gene_sequence, alignment_name = process_mafft_output(mafft_run_folder, error_exon)

                                print(f"Original query = {original_query}")
                                print(f"New query = {possible_gene_sequence}")
                                
                                list_of_folders_inside_exon_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}")
                                if "for_blast" not in list_of_folders_inside_exon_folder:
                                    os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast")
                                else:
                                    list_of_files = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast")
                                    for file in list_of_files:
                                        os.remove(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast/{file}")


                                with open(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast/new_query.txt",'w') as query_file:
                                    query = f">Query_{error_exon}_{alignment_name}\n{possible_gene_sequence}"
                                    query_file.write(query)
                    

Scaffold = CAKASE010000043.1, start = 6369470, end = 6369558
Danaus_chrysippus,CAKASE010000043.1,6369627,6369710,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_1_rightoh_2_query_Exon_2,1,27,27

1
4
Danaus_chrysippus,CAKASE010000043.1,6369470,6369558,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_2_query_Exon_1,1,29,29

Danaus_chrysippus,CAKASE010000043.1,6370670,6370764,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_2_rightoh_0_query_Exon_4,1,31,31

Getting Gene
min = set4_frame2, 0.313
5 top scores:
[['set4_frame2', 0.313], ['set1_frame1', 1.693], ['set38_frame1', 1.711], ['set30_frame2', 1.775], ['set16_frame0', 1.782]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Danaus_chrysippus/Period_gene_genomic_sequence_individual_exon_29.Nymphalis_io/Exon_2/for_alignment/alignment_Exon_2_translated_genomic_sequence_4_frame2.fa.txt
29 56
29.Nymphalis_io_XM_050504608.1_Frame_1_rightoh_2_query_Exon_2
YSSKSMHSGSLSSGSSGYGGKPSTSSY


Getting Gene
min = set46_frame1, 0.354
5 top scores:
[['set19_frame1', 1.857], ['set37_frame1', 1.838], ['set46_frame1', 0.354]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Danaus_chrysippus/Period_gene_genomic_sequence_individual_exon_29.Nymphalis_io/Exon_9/for_alignment/alignment_Exon_9_translated_genomic_sequence_46_frame1.fa.txt
9 82
29.Nymphalis_io_XM_050504608.1_Frame_2_rightoh_2_query_Exon_9
EVIVNAKPFVIRHSASGSLEFIDTESVPYLGYLPQDILKKNALQLYHPADLGYVRQIYESIVKEGVVERSKPY
9 82
set46_frame1
EIIHNAIPFVIRHSATGSLEYIDHESVPYLGYLPQDIVEKDALQLYHPGDLGYLRQIYETIVKEGGVQRSKPY
Original query = PNEVIVNAKPFVIRHSASGSLEFIDTESVPYLGYLPQDILKKNALQLYHPADLGYVRQIYESIVKEGVVERSKPY
New query = EIIHNAIPFVIRHSATGSLEYIDHESVPYLGYLPQDIVEKDALQLYHPGDLGYLRQIYETIVKEGGVQRSKPY
Scaffold = CAKASE010000043.1, start = 6369470, end = 6369558
Danaus_chrysippus,CAKASE010000043.1,6374739,6374850,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_1_rightoh_0_query_Exon_10,1,37

Getting Gene
min = set28_frame1, 0.995
5 top scores:
[['set38_frame0', 1.852], ['set28_frame1', 0.995], ['set33_frame1', 1.781], ['set14_frame2', 1.878]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Danaus_chrysippus/Period_gene_genomic_sequence_individual_exon_29.Nymphalis_io/Exon_16/for_alignment/alignment_Exon_16_translated_genomic_sequence_28_frame1.fa.txt
28 85
29.Nymphalis_io_XM_050504608.1_Frame_1_rightoh_0_query_Exon_16
HNADMEKELVKRHRDNRICSKFERDILSSEMKQKKKEHLARCNAFIQQSILENAEQK
28 85
set28_frame1
HNAEMERELIKMYRENR-SSKGDREKASNETRQKKKQHLARCNAAFQPTSLGLSDPQ
Original query = HNADMEKELVKRHRDNRICSKFERDILSSEMKQKKKEHLARCNAFIQQSILENAEQK
New query = HNAEMERELIKMYRENR-SSKGDREKASNETRQKKKQHLARCNAAFQPTSLGLSDPQ
Scaffold = CAKASE010000043.1, start = 6369470, end = 6369558
Danaus_chrysippus,CAKASE010000043.1,6379020,6379164,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_1_query_Exon_17,4,48,48

15
19
Danaus_chrysippus,CAKASE0

Getting Gene
min = set49_frame0, 1.486
5 top scores:
[['set58_frame0', 1.679], ['set22_frame1', 1.752], ['set23_frame1', 1.761], ['set56_frame1', 1.765], ['set49_frame0', 1.486]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Danaus_chrysippus/Period_gene_genomic_sequence_individual_exon_29.Nymphalis_io/Exon_23/for_alignment/alignment_Exon_23_translated_genomic_sequence_49_frame0.fa.txt
2 44
29.Nymphalis_io_XM_050504608.1_Frame_0_rightoh_0_query_Exon_23
SWQKQNGSGDAISFQSFK--VLNGETKKKSLRSNIEPPWTKQ
2 44
set49_frame0
SWHKNQKGDDLMSLQSSTEAVTYTPNKGQVQQKKVDPSWVEE
Original query = SWQKQNGSGDAISFQSFKVLNGETKKKSLRSNIEPPWTKQ
New query = SWHKNQKGDDLMSLQSSTEAVTYTPNKGQVQQKKVDPSWVEE
Scaffold = CAKASE010000043.1, start = 6369470, end = 6369558
Danaus_chrysippus,CAKASE010000043.1,6383278,6383379,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_0_query_Exon_24,1,34,34

22
25
Danaus_chrysippus,CAKASE010000043.1,6382357,6382515,0,N,13.Danau

Getting Gene
min = set16_frame2, 0.402
5 top scores:
[['set16_frame2', 0.402], ['set59_frame2', 1.53], ['set64_frame2', 1.696], ['set6_frame0', 1.701], ['set10_frame2', 1.722]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Danaus_plexippus/Period_gene_genomic_sequence_individual_exon_29.Nymphalis_io/Exon_7/for_alignment/alignment_Exon_7_translated_genomic_sequence_16_frame2.fa.txt
18 43
29.Nymphalis_io_XM_050504608.1_Frame_0_rightoh_1_query_Exon_7
DRNTFASQITNGLTVPKNANGMQIK
18 43
set16_frame2
DRNTFASQITSGLAVPKNVNGTQEK
Original query = DRNTFASQITNGLTVPKNANGMQIK
New query = DRNTFASQITSGLAVPKNVNGTQEK
Scaffold = NC_083559.1, start = 6597896, end = 6597984
Danaus_plexippus,NC_083559.1,6602103,6602306,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_2_rightoh_1_query_Exon_8,1,67,67

6
10
Danaus_plexippus,NC_083559.1,6599466,6599597,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_0_query_Exon_6,1,44,44

Danaus_plexippus,NC_08355

...

Getting Gene
min = set7_frame0, 1.448
5 top scores:
[['set38_frame2', 1.797], ['set7_frame0', 1.448], ['set19_frame0', 1.835], ['set62_frame2', 1.838], ['set37_frame2', 1.863]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Melinaea_menophilus/Period_gene_genomic_sequence_individual_exon_29.Nymphalis_io/Exon_3/for_alignment/alignment_Exon_3_translated_genomic_sequence_7_frame0.fa.txt
6 61
29.Nymphalis_io_XM_050504608.1_Frame_1_rightoh_1_query_Exon_3
HAIQLSEKRMKDREVKKKKKMQIVSKMKKDKYVVVQSLPQLE-PVQEITTEYGNN
6 61
set7_frame0
NLSQPPEKWLKEKESKKKKPVQVELKPAEEK---VEECPQEPIPVFETMEEETNN
Original query = HAIQLSEKRMKDREVKKKKKMQIVSKMKKDKYVVVQSLPQLEPVQEITTEYGNN
New query = NLSQPPEKWLKEKESKKKKPVQVELKPAEEK---VEECPQEPIPVFETMEEETNN
Scaffold = OU911050.1, start = 15742387, end = 15742475
Melinaea_menophilus,OU911050.1,15740654,15740748,1,N,13.Danaus_plexippus_XM_061526465.1_Frame_2_rightoh_0_query_Exon_4,1,31,31

2
6
Melinaea_menophilus,OU91105

Getting Gene
min = set20_frame0, 0.328
5 top scores:
[['set1_frame2', 1.686], ['set38_frame2', 1.758], ['set65_frame2', 1.788], ['set20_frame0', 0.328], ['set22_frame0', 1.771]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Melinaea_menophilus/Period_gene_genomic_sequence_individual_exon_29.Nymphalis_io/Exon_10/for_alignment/alignment_Exon_10_translated_genomic_sequence_20_frame0.fa.txt
8 45
29.Nymphalis_io_XM_050504608.1_Frame_1_rightoh_0_query_Exon_10
VLTQNGDYLKLVTEWSSFINPWSRKLEFVTAKHFIIE
8 45
set20_frame0
MMAQNGDYLRLETEWSSFINPWSKKLEFVIGKHYIIE
Original query = VLTQNGDYLKLVTEWSSFINPWSRKLEFVTAKHFIIE
New query = MMAQNGDYLRLETEWSSFINPWSKKLEFVIGKHYIIE
Scaffold = OU911050.1, start = 15742387, end = 15742475
Melinaea_menophilus,OU911050.1,15737330,15737449,1,N,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_0_query_Exon_11,1,40,40

9
13
Melinaea_menophilus,OU911050.1,15737701,15737929,1,N,13.Danaus_plexippus_XM_061526465.1_Fr

min = set91_frame0, 1.047
5 top scores:
[['set5_frame1', 1.558], ['set4_frame2', 1.589], ['set61_frame2', 1.731], ['set91_frame0', 1.047], ['set98_frame0', 1.789]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Melinaea_menophilus/Period_gene_genomic_sequence_individual_exon_29.Nymphalis_io/Exon_17/for_alignment/alignment_Exon_17_translated_genomic_sequence_91_frame0.fa.txt
28 70
29.Nymphalis_io_XM_050504608.1_Frame_0_rightoh_1_query_Exon_17
PQGVKRTSRQKEEGSAQKHRCSSARLKRQKQIVRAANSQSNS
28 70
set91_frame0
PHGLKRPSKQTEEASAHKHRCSSPRQSRPHSIPSNVPQSNNV
Original query = PQGVKRTSRQKEEGSAQKHRCSSARLKRQKQIVRAANSQSNSTM
New query = PHGLKRPSKQTEEASAHKHRCSSPRQSRPHSIPSNVPQSNNV
Scaffold = OU911050.1, start = 15742387, end = 15742475
Melinaea_menophilus,OU911050.1,15729969,15730076,1,N,13.Danaus_plexippus_XM_061526465.1_Frame_2_rightoh_1_query_Exon_18,2,35,35

16
20
Melinaea_menophilus,OU911050.1,15733569,15733740,1,N,13.Danaus_plexippus_XM_061

min = set51_frame2, 0.644
5 top scores:
[['set51_frame2', 0.644], ['set14_frame2', 1.66], ['set11_frame0', 1.731], ['set29_frame2', 1.74], ['set63_frame2', 1.759]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Melinaea_menophilus/Period_gene_genomic_sequence_individual_exon_29.Nymphalis_io/Exon_24/for_alignment/alignment_Exon_24_translated_genomic_sequence_51_frame2.fa.txt
9 43
29.Nymphalis_io_XM_050504608.1_Frame_0_rightoh_0_query_Exon_24
VRVTSELIYKYQICTKDMDEVLRCDKEKLEKLEQ
9 43
set51_frame2
VCVTSELIYRYQIRTKNLDEVLSEDKKKLETLEQ
Original query = VRVTSELIYKYQICTKDMDEVLRCDKEKLEKLEQ
New query = VCVTSELIYRYQIRTKNLDEVLSEDKKKLETLEQ
Scaffold = OU911050.1, start = 15742387, end = 15742475
Melinaea_menophilus,OU911050.1,15724080,15724220,1,Y,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_0_query_Exon_25,1,26,47

23
25
26
Melinaea_menophilus,OU911050.1,15725100,15725225,1,N,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_0_query_

Getting Gene
min = set84_frame2, 0.619
5 top scores:
[['set84_frame2', 0.619], ['set117_frame0', 1.852], ['set82_frame2', 1.852], ['set32_frame1', 1.865], ['set60_frame1', 1.851]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Napeogenes_inachia/Period_gene_genomic_sequence_individual_exon_29.Nymphalis_io/Exon_8/for_alignment/alignment_Exon_8_translated_genomic_sequence_84_frame2.fa.txt
16 85
29.Nymphalis_io_XM_050504608.1_Frame_2_rightoh_1_query_Exon_8
TQSGTGNQTSTMVCRIRRYRGLQTGFRIKEKILTYMPFLLKFFFKNVVHDESHVVYLVIQATPYYSAFK
16 85
set84_frame2
TAPVPGNHISTMVCRIRRYRGLNIGFGIKDKSVSFMPFLLKFFFKNINDEEGQVIYLVIQATPFFSAFK
Original query = ATQSGTGNQTSTMVCRIRRYRGLQTGFRIKEKILTYMPFLLKFFFKNVVHDESHVVYLVIQATPYYSAFK
New query = TAPVPGNHISTMVCRIRRYRGLNIGFGIKDKSVSFMPFLLKFFFKNINDEEGQVIYLVIQATPFFSAFK
Scaffold = OY365800.1, start = 4774014, end = 4774102
Napeogenes_inachia,OY365800.1,4781546,4781774,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_2_righ